En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
#!pip3 install -r ../requirements.txt

In [2]:
from cProfile import Profile
from pstats import SortKey, Stats
from q1_memory import q1_memory
from q1_time import q1_time
from memory_profiler import memory_usage
from statistics import mean
import pprint
from datetime import datetime

In [3]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [4]:

with Profile() as profile:
    RS = q1_memory(file_path)
    stats = Stats(profile)
    stats.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS)
total_time = sum(stat[2] for stat in stats.stats.values())
#mem_used = mean(memory_usage((q1_memory,(),{'file_path':file_path}),interval=0.1))
#print("\nmemory", mem_used)
print("time", total_time)

with Profile() as profile:
    RS2 = q1_time(file_path)
    stats2 = Stats(profile)
    stats2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS2)
#mem_used2 = mean(memory_usage((q1_time,(),{'file_path':file_path}),interval=0.1))
total_time2 = sum(stat[2] for stat in stats2.stats.values())
#print("\nmemory", mem_used2)
print("time", total_time2)
assert(RS == RS2)

[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 19, 0, 0), 'Preetm91'),
 (datetime.datetime(2021, 2, 20, 0, 0), 'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0), 'Surrypuria')]
time 4.397011449
[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.dateti

In [5]:
from cProfile import Profile
from pstats import SortKey, Stats
from q2_memory import q2_memory
from q2_time import q2_time
from memory_profiler import memory_usage
from statistics import mean

file_path = "farmers-protest-tweets-2021-2-4.json"
with Profile() as profile:
    RS = q2_memory(file_path)
    stats = Stats(profile)
    stats.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS)
total_time = sum(stat[2] for stat in stats.stats.values())
mem_used = mean(memory_usage((q2_memory,(),{'file_path':file_path}),interval=0.1))
print("\nmemory", mem_used)
print("time", total_time)

with Profile() as profile:
    RS2 = q2_time(file_path)
    stats2 = Stats(profile)
    stats2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS2)
mem_used2 = mean(memory_usage((q2_time,(),{'file_path':file_path}),interval=0.1))
total_time2 = sum(stat[2] for stat in stats2.stats.values())
print("\nmemory", mem_used2)
print("time", total_time2)
assert(RS == RS2)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

memory 127.42312409156976
time 30.867466067000006
[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

memory 499.89284393491124
time 16.664878012000003


In [6]:
from cProfile import Profile
from pstats import SortKey, Stats
from q3_memory import q3_memory
from q3_time import q3_time
from memory_profiler import memory_usage
from statistics import mean

file_path = "farmers-protest-tweets-2021-2-4.json"
with Profile() as profile:
    RS = q3_memory(file_path)
    stats = Stats(profile)
    stats.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS)
total_time = sum(stat[2] for stat in stats.stats.values())
mem_used = mean(memory_usage((q3_memory,(),{'file_path':file_path}),interval=0.1))
print("\nmemory", mem_used)
print("time", total_time)

with Profile() as profile:
    RS2 = q3_time(file_path)
    stats2 = Stats(profile)
    stats2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

pprint.pprint(RS2)
mem_used2 = mean(memory_usage((q3_time,(),{'file_path':file_path}),interval=0.1))
total_time2 = sum(stat[2] for stat in stats2.stats.values())
print("\nmemory", mem_used2)
print("time", total_time2)
assert(RS == RS2)

[('NARENDRAMODI', 2266),
 ('KISANEKTAMORCHA', 1840),
 ('RAKESHTIKAITBKU', 1642),
 ('PMOINDIA', 1427),
 ('RAHULGANDHI', 1146),
 ('GRETATHUNBERG', 1048),
 ('RAVISINGHKA', 1019),
 ('RIHANNA', 986),
 ('UNHUMANRIGHTS', 962),
 ('MEENAHARRIS', 926)]

memory 167.27118055555556
time 4.941426472
[('NARENDRAMODI', 2266),
 ('KISANEKTAMORCHA', 1840),
 ('RAKESHTIKAITBKU', 1642),
 ('PMOINDIA', 1427),
 ('RAHULGANDHI', 1146),
 ('GRETATHUNBERG', 1048),
 ('RAVISINGHKA', 1019),
 ('RIHANNA', 986),
 ('UNHUMANRIGHTS', 962),
 ('MEENAHARRIS', 926)]

memory 471.72977430555557
time 4.255958143


In [7]:
%%time
import pandas as pd

# Leo el archivo, le indico que cada linea es un json, y que las fechas las deje en string
df = pd.read_json(file_path, lines=True, convert_dates=False)

# Selecciono solamente las dos columnas que necesito, lo demás se pueder ir para liberrar memoria
df_selected = df[['date', 'user']]

# Elimino el dataframe original, ya que es excesivamente grande
del df

# Aplico la transformación del usuario, obtengo el username y me deshago de todo lo dem
df_selected['user'] = df_selected['user'].apply(lambda x: x['username'])

# Aplico la transformación de la fecha, para quedarme solo con AAAA-MM-DD, esto facilita los agrupamientos
df_selected['date'] = df_selected['date'].apply(lambda x: x[:10])

df_selected.head()

CPU times: user 8.61 s, sys: 1.94 s, total: 10.5 s
Wall time: 9.96 s


,date,user
0,2021-02-24,ArjunSinghPanam
1,2021-02-24,PrdeepNain
2,2021-02-24,parmarmaninder
3,2021-02-24,anmoldhaliwal
4,2021-02-24,KotiaPreet


In [33]:
%%time

# Primera Agrupación de conteo por fecha. Esto nos da el conteo de tweets cada día.
# Observe que se resetean los indices para que el agrupamiento de conteo se vuelva una columna llamada "count"
group = df_selected.groupby(["date"]).size().reset_index(name="count").nlargest(10, "count")

# Se filtra el df principal para deshacernos de las fechas que ya no están en el top
df_selected = df_selected[df_selected["date"].isin(group["date"])]
del group

# Segunda Agrupación de conteo por fecha y usuario al tiempo. Esto nos da el conteo de cada usuario en cada día.
# Observe que se resetean los idices para volver a un dataframe con solo dos indices, el agrupamiento de conteo se vuelve una columna llamada "count"
group2 = df_selected.groupby(["date","user"]).size().reset_index(name='count')

# Ya teniendo el conteo de cada usuario en cada día, volvemos a agrupar por fecha, esta vez quedadoos con aque usuario que tuvo más conteo, es decir, aquel que hizo más publicacioes cada día
group2 = group2.groupby("date").apply(lambda groupdate: groupdate.nlargest(1, 'count'))

# este agrupamiento nos da el usuario más activo cada dia, juto con el indice en el que se encontraba
group2 = group2.reset_index(drop=True)
group2
pprint.pprint([(datetime.fromisoformat(group2["date"][i]), group2["user"][i]) for i in range(len(group2))])

[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 19, 0, 0), 'Preetm91'),
 (datetime.datetime(2021, 2, 20, 0, 0), 'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0), 'Surrypuria')]
CPU times: user 142 ms, sys: 8.34 ms, total: 151 ms
Wall time: 150 ms


<timed exec>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
